In [26]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import os
from urllib import parse
import numpy as np
import pymysql.cursors

#대중 리뷰
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

네이버 영화 메인 페이지 1~10위 영화 상세페이지 주소 가져오기

In [27]:
def step1_get_detail_url() :
    # 접속할 페이지의 주소
    site = 'https://movie.naver.com/movie/running/current.nhn?order=reserve'

    # 접속한다.
    response = requests.get(site)   

    bs = BeautifulSoup(response.content, 'html.parser')
    #print(bs)

    # a 태그들을 가져온다.
    a_list = bs.select('.top_thumb_lst a')

    # href 속성을 가져온다.
    df = pd.DataFrame()
    for idx in range(10) :       # 상위 20개만 가져오기
        href = a_list[idx].get('href')
        
        # 가져온 href 속성의 주소를 분석한 객체를 생성한다.
        a1 = parse.urlparse(href)
        
        # 주소를 분석한 객체서 쿼리 스트링을 가져온다
        query_str = parse.parse_qs(a1.query)
        
        # 추출한 쿼리스트링 데이터에서 원하는 파라미터 데이터를 추출한다.
        code = query_str['code'][0]
        

        # print(href)
        df = df.append([[code]], ignore_index=True)

    df.columns = ['code']
    df.to_csv('movie_code_list.csv', index=False, encoding='utf-8-sig')
    print('주소 저장 완료')

In [28]:
step1_get_detail_url()

주소 저장 완료


평점 더보기 주소 스크랩

In [29]:
def step2_get_reple_href() :
    # 스크랩한 영화 코드를 불러온다.
    code_frame = pd.read_csv('movie_code_list.csv')
    code_list = code_frame['code'].tolist()
    # 테스트용
 #   code_list = ['156464', '109906']


    # 영화코드와 주소를 합쳐서 요청할 주소를 만든다.
    url_list = pd.DataFrame()
    for code in code_list:
        site = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=%s&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false' % code
        # print(site)
        url_list = url_list.append([[site]], ignore_index=True)

    url_list.columns = ['url']
    url_list.to_csv('movie_url_list.csv', index=False, encoding='utf-8-sig')
    print('저장완료')
    

In [30]:
step2_get_reple_href()

저장완료


예매순 1~10 영화의 1페이지부터 마지막페이지 까지 순회하면서 평점과 리뷰 데이터 가져와서 저장

In [50]:
def step3_get_reply_data() :
    # csv에 저장되어 있는 url 데이터를 가져온다.
    df = pd.read_csv('movie_url_list.csv')
    url_list = df['url'].tolist()

    i=1 #영화 순위 카운팅
#     result_df = pd.DataFrame()
    
    for url in url_list :
        print(url)
        # 해당 영화의 첫 페이지 html 데이터를 가져온다. (총 몇건의 리뷰가 있는지 확인해서 "페이지수"를 계산하기 위함)
        response = requests.get(url)
        bs = BeautifulSoup(response.content, 'html.parser')
        # print(bs)

        # 총 페이지 수를 구한다.
        strong = bs.select('.total em')
        #1->0
        score_total = int(strong[0].text.replace(',', ''))    # 쉼표 없애기 / int(정수형)로 만들기
        # print(score_total)
        pageCnt = score_total // 10     # 한페이지당 10개의 리뷰가 있어서
        if score_total % 10 > 0 :
            pageCnt += 1
        print(pageCnt)

        # 전체 페이지를 돌면서 140평 데이터를 가져온다.
        # 현재 페이지
        now_page = 1

#         pageCnt = 5        # 테스트로 5페이지까지만
        
        while now_page <= pageCnt :
            #sleep(1)
            #sleep(0.0001)
            # 요청할 페이지의 주소
            url2 = url + '&page=' + str(now_page)
            # print(url2)

            # 140자평 데이터를 추출한다.
            response2 = requests.get(url2)
            bs2 = BeautifulSoup(response2.content, 'html.parser')
            

            
#             result_df2 = pd.DataFrame()

            # li 태그들을 가져온다.(score_reple 태그-리뷰-를 포함하고 있는)
            lis = bs2.select('.score_result li')

            for obj in lis :
                
                result_df = pd.DataFrame()
                # 평점
                star_score = obj.select('.star_score em')[0].text
                # 140자평
                score_reple = obj.select('.score_reple p')[0].text
                # 아이디
                reviewer = obj.select('.score_reple em')[0].text
                # 날짜
#                 date = obj.select('.score_reple em')[1].text            
                # print(star_score)
                # print(score_reple)

                # 저장한다.
                r_df = result_df.append([[score_reple, star_score, reviewer , i]], ignore_index =True)
#                 result_df2 = result_df2.append([[reviewer, date, i]], ignore_index=True)                
#             result_df.columns = ['text', 'score','reviewer', 'date', 'movie_id']
#               result_df.columns = ['text', 'score']
#             print(result_df)

            if os.path.exists('star_score.csv') == False :     # 아직 파일이 없으면 파일을 만든다.
                r_df.columns = ['text', 'score','reviewer','movie_id']
#                 result_df2.columns = ['reviewer', 'date', 'movie_id']                                              
                r_df.to_csv('star_score.csv', index=False, encoding='utf-8-sig')
#                 result_df2.to_csv('star_score2.csv', index=False, encoding='utf-8-sig')                              
            else :                                               # 이미 파일이 있으면 결과를 더한다.
                r_df.to_csv('star_score.csv', index=False, encoding='utf-8-sig', mode='a', header=False)
#                 result_df2.to_csv('star_score.csv', index=False, encoding='utf-8-sig', mode='a', header=False)
            print("%d / %d" % (now_page, pageCnt))               # 진행경과를 보여준다. n번째 중 몇 번째 진행중인지.
            now_page += 1
        i=i+1 #movie_id 값 1증가
#     result_df.columns = ['text', 'score','reviewer', 'date', 'movie_id']
    print('저장완료')
#     return result_df

In [51]:
step3_get_reply_data()


https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=106360&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false
1436
1 / 1436
2 / 1436
3 / 1436
4 / 1436
5 / 1436
6 / 1436
7 / 1436
8 / 1436
9 / 1436
10 / 1436
11 / 1436
12 / 1436
13 / 1436
14 / 1436
15 / 1436
16 / 1436
17 / 1436
18 / 1436
19 / 1436
20 / 1436
21 / 1436
22 / 1436
23 / 1436
24 / 1436
25 / 1436
26 / 1436
27 / 1436
28 / 1436
29 / 1436
30 / 1436
31 / 1436
32 / 1436
33 / 1436
34 / 1436
35 / 1436
36 / 1436
37 / 1436
38 / 1436
39 / 1436
40 / 1436
41 / 1436
42 / 1436
43 / 1436
44 / 1436
45 / 1436
46 / 1436
47 / 1436
48 / 1436
49 / 1436
50 / 1436
51 / 1436
52 / 1436
53 / 1436
54 / 1436
55 / 1436
56 / 1436
57 / 1436
58 / 1436
59 / 1436
60 / 1436
61 / 1436
62 / 1436
63 / 1436
64 / 1436
65 / 1436
66 / 1436
67 / 1436
68 / 1436
69 / 1436
70 / 1436
71 / 1436
72 / 1436
73 / 1436
74 / 1436
75 / 1436
76 / 1436
77 / 1436
78 / 1436
79 / 1436
80 / 1436
81 / 1436
82 / 1436
8

IndexError: list index out of range

In [33]:
def prepro():
    df = pd.read_csv('star_score.csv')
    df.columns = ["리뷰","평점","작성자","movie_id"]

    df['리뷰']=df['리뷰'].str.replace(pat='관람객', repl='', regex=False)
    df[df['리뷰'].str.contains("스포일러가 포함된 감상평입니다. 감상평 보기")]= np.nan #nan으로 변경
    df['리뷰']= df['리뷰'].str.strip()  # 앞 뒤 공백을 제거
    df['작성자']= df['작성자'].str.strip()  # 앞 뒤 공백을 제거

    df=df.replace('',np.NaN)
    result=df.dropna()    
#     df.dropna()
    final_result=result.reset_index(drop=True)
    return final_result

In [34]:
df=prepro()

In [35]:
df.head(50)

,리뷰,평점,작성자,movie_id
0,라라랜드 보다 재밌다. 10만원이 넘는 비싼 뮤지컬 보다 1만원이 안되는 위대한 쇼...,10.0,라라랜드 보다 재밌다. 10만원이 넘는 비싼 뮤지컬 보다 1만원이 안되는 위대한 쇼...,1.0
1,점왜 이영화 말이 없나 했더니 .. 이런 반전이 .. 졸라 재밌어,10.0,점왜 이영화 말이 없나 했더니 .. 이런 반전이 .. 졸라 재밌어,1.0
2,영화보다 박수칠뻔했습니다,10.0,짱구(hyeo****),1.0
3,뭔가 나도 모르게 엔딩에서 박수침 ㅋ 한편의 뮤지컬을 본듯,10.0,엄마칙까칙까해조(nice****),1.0
4,휴잭맨은 정말 빛이났고 한명도 묻히는 캐릭터없이각자의 역할이 다 빛났어요 보통 뮤지...,10.0,휴잭맨은 정말 빛이났고 한명도 묻히는 캐릭터없이각자의 역할이 다 빛났어요 보통 뮤지...,1.0
5,재미있게 보고 나온 후 알게된 사실은 영화의 실제 주인공인 P.T. 은 영화와는 정...,1.0,재미있게 보고 나온 후 알게된 사실은 영화의 실제 주인공인 P.T. 은 영화와는 정...,1.0
6,1점도 아깝다. 휴잭맨의 미스 초이스. 실제 프릭쇼를 미화하고 단원들은 그저 사업가...,1.0,1점도 아깝다. 휴잭맨의 미스 초이스. 실제 프릭쇼를 미화하고 단원들은 그저 사업가...,1.0
7,연말에 어울리는 마음이 따뜻해지는 영화였어요.노래가 정말 너무 좋아요. 강추,10.0,연말에 어울리는 마음이 따뜻해지는 영화였어요.노래가 정말 너무 좋아요. 강추,1.0
8,"노래는 엄청 좋지만, 전개가 너무 빨라서 당황스러움",7.0,FSN(smh1****),1.0
9,영화값으로 볼수없는 위대한 쇼를 봤어요 행복합니당,10.0,마들레몬(joyn****),1.0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pickle
from konlpy.tag import *

In [ ]:
# 형태소 분석을 위한 함수
def tokenizer(text) :
    okt = Okt()
    return okt.morphs(text)

In [ ]:
df.head(50)

In [ ]:
def step7_insert_review(df) :
    # 객체를 복원한다.
    with open('pipe.dat', 'rb') as fp:
        pipe = pickle.load(fp)

    import numpy as np
    import pandas as pd
    
    
    rlist=df["리뷰"].tolist()
    df=df.drop("리뷰",axis=1) 
    df_result = pd.DataFrame()
    
    for i in rlist :
        text = i

        str = [text]
        # 예측 정확도
        r1 = np.max(pipe.predict_proba(str) * 100)
        # 예측 결과
        r2 = pipe.predict(str)[0]

        if r2 == '1' :
            print(text)
            
            print('긍정적인 리뷰')
            df_result = df_result.append([[text, r1, r2]],ignore_index=True)
            print('----------------------------')
        else :
            print(text)
            print('부정적인 리뷰')
            df_result = df_result.append([[text, r1, r2]],ignore_index=True)
            print('----------------------------')

        print('정확도 : %.3f' % r1)
    
    df_result.columns = ['리뷰','정확도','결과']
    final_result=pd.concat([df_result,df],axis=1)
    return final_result


In [ ]:
final_result = step7_insert_review(df)

In [ ]:
final_result.head(50)  #1 긍정, 0 부정

대중적인 리뷰 시작
--------------------------------------

In [ ]:
import sister

In [ ]:
sentence_embedding = sister.MeanEmbedding(lang="file",fasttextfile="cc.ko.100.bin")

In [ ]:
final_result['clustering']=0

In [ ]:
final_result.head(49)

In [ ]:
movie_id_count=pd.DataFrame(final_result['movie_id'].value_counts().sort_index())

In [ ]:
movie_id_count

In [ ]:
movie_id_count=movie_id_count['movie_id'].tolist()

In [ ]:
movie_id_count

In [ ]:
def count_loop(c_list):
    c_list.insert(0,0)  #리스트 맨 앞에 0 넣기
    c_list.pop()        #리스트 마지막 값 빼기
    print(len(c_list))
    a=[]
    temp=0
    for i in range(0,10): #10개 영화
        temp+=c_list[i]
        a.append(temp)

    return a      #결과로 나온 각 영화에 대한 index 값에 더하기 위해

In [ ]:
a=count_loop(movie_id_count)

In [ ]:
def separate(df,count_list): #각 영화 리뷰 2차원 배열로 분류
    review=df['리뷰'].tolist()
    total = len(count_list)
    sample=[]
    current=0
    for i in range(total):

        temp=[]
        print('----------------------------')
        print(i)
        print("영화 번호")
        print('----------------------------')
        for j in range(count_list[i]):
            print(current+j)
            temp.append(review[current+j])
        sample.append(temp)
        current += count_list[i]
    return sample        

In [ ]:
sep_review=separate(final_result,movie_id_count)

In [ ]:
sep_review

In [ ]:
def review_embedding(review_list): #각 영화 리뷰를 100차원의 값으로 임베딩
    vector1_list = []
    for i in range(len(review_list)): 
    
        s1 = sentence_embedding(review_list[i])
        vector1_list.append(s1)
        
    vector_df= pd.DataFrame(vector1_list) #DataFrame으로
    return vector_df #merge

In [ ]:
def model(X):
    ifor = IsolationForest(n_estimators=100, max_samples=512,contamination=0.0235).fit(X) #모델 생성  
    return ifor

In [ ]:
def d_scatter(scores_f):
    for i in range(10):
            plt.scatter(scores_f[i],1,color='red',marker='*') #scores_f 값을 *으로 마킹. 시각화

In [ ]:
def outlier_comp(data):
#     c_data= concat(data)
    ifor=model(data)
    scores_f= -ifor.decision_function(data)
    d_scatter(scores_f)
    return scores_f

In [ ]:
def min_score(score): #outlier score 최소인 10개 값 추출
    hi=score.tolist()
    hi.sort()
    a=hi[:10]
    return a

In [ ]:
def top_review(score,value,review): #Outlit score 최소인 20개 리뷰 확인
    index = []
    top20_review=[]
    score=score.tolist()
    for i in range(len(value)):
        index.append(score.index(value[i]))
    
    for ind in index:
        top20_review.append(review[ind])
        
    return top20_review

In [ ]:
def index(score,value,m_count): #Outlit score 최소인 20개 리뷰 확인
    index = []
    top20_review=[]
    score=score.tolist()
    for j in range(10): #10개 추출
        find_index=score.index(value[j])
        f_index=find_index+m_count #각 영화로 인덱스 맞춤
        index.append(f_index)
    return index

In [ ]:
def df_to_list(df):
    review=final_result['리뷰'].tolist()
    accuracy=final_result['정확도'].tolist()
    result=final_result['결과'].tolist()
    # score=final_result['평점'].tolist()
    reviewer=final_result['작성자'].tolist()
    date=final_result['날짜'].tolist()
    movie_id=final_result['movie_id'].tolist()
    clustering=final_result['clustering'].tolist()
    return review, accuracy, result, reviewer, date, movie_id, clustering

In [ ]:
def db_connect(final_result):
    review, accuracy, result, reviewer, date, movie_id, clustering = df_to_list(final_result)

    conn = pymysql.connect( #DB연결
    host='localhost',
    user='root',
    password='1234',
    db='movie_review',
    charset='utf8' 
    )
    total = len(review) #리뷰 개수
    curs = conn.cursor()
    sql = "insert into reviews(review_id, review_movie_id, review_contents, review_accuracy, review_result, review_reviewer, review_date, review_clustering) values(%s,%s,%s,%s,%s,%s,%s,%s)"
    for i in range(total):
        curs.execute(sql, (i+1 ,movie_id[i], review[i], accuracy[i], result[i], reviewer[i], date[i], clustering[i]))
        print('ok')    
        conn.commit()

In [ ]:
movie_id_count

In [ ]:
m_count=count_loop(movie_id_count)

In [ ]:
movie_id_count

In [ ]:
m_count

In [ ]:
ind_list=[]
for i in range(10): #영화 10개
    df=review_embedding(sep_review[i])
    score_s=outlier_comp(df)
    min_value=min_score(score_s)
    ind = index(score_s, min_value, m_count[i])
    for x in ind:
        final_result['clustering'][x]=1

In [ ]:
final_result

In [ ]:
db_connect(final_result)

In [ ]:
movie_id_count=pd.DataFrame(final_result['movie_id'].value_counts().sort_index())
movie_id_count=movie_id_count['movie_id'].tolist()

In [ ]:
movie_id_count

In [ ]:
movie_id_count.insert(0,0)
movie_id_count.pop()

In [ ]:
movie_id_count

In [ ]:
m_count=count_loop(movie_id_count)

In [ ]:
len(m_count)

In [ ]:
ind = index(score_s, min_value, m_count)

In [ ]:
final_result

In [ ]:
for x in ind:
    final_result['clustering'][x]=1

In [ ]:
final_result

In [ ]:
def clustering(df,sep_review,movie_id_count):
    finish=pd.DataFrame()
    sample=[]
    for i in range(10):
        df=review_embedding(sep_review[i])
        score_s=outlier_comp(df)
        min_value=min_score(score_s)
        ind = index(score_s, min_value, movie_id_count[i])
    # top=top_review(min_score,)
#         finish=pd.concat([finish,df], ignore_index=True)
        sample.append(ind)
#         for x in ind:
#             df['clustering'][x]=1
    return sample
#     return ind

In [ ]:
movie_id_count[9]

In [ ]:
movie_id_count

In [ ]:
m_count=count_loop(movie_id_count)

In [ ]:
f_result=clustering(final_result,sep_review,m_count)

In [ ]:
f_result[5]

In [ ]:
clustering

In [ ]:
# finish=pd.DataFrame()
# for i in range(10):
#     df=review_embedding(asdf[i])
#     score_s=outlier_comp(df)
#     min_value=min_score(score_s)
#     ind = index(score_s,min_value,ss[i])
# # top=top_review(min_score,)
# #     finish=pd.concat([finish,df], ignore_index=True)
#     for x in ind:
#         final_result['clustering'][x]=1

In [ ]:
final_result

In [ ]:
import sister

In [ ]:
sentence_embedding = sister.MeanEmbedding(lang="file",fasttextfile="cc.ko.100.bin")

In [ ]:
def review_embedding(df): #리뷰를 100차원의 값으로 임베딩
    review_list=df['리뷰'].tolist()
    vector1_list = []
    for i in range(len(review_list)): 
    
        s1 = sentence_embedding(review_list[i])
        vector1_list.append(s1)
        
        
    vector_df= pd.DataFrame(vector1_list) #DataFrame으로
    return vector_df #merge

In [ ]:
r_embedding=review_embedding(final_result)

In [ ]:
r_embedding

In [ ]:
def model(X):
    ifor = IsolationForest(n_estimators=100, max_samples=512,contamination=0.0235).fit(X) #모델 생성  
    return ifor

In [ ]:
def d_scatter(scores_f):
    for i in range(10):
            plt.scatter(scores_f[i],1,color='red',marker='*') #scores_f 값을 *으로 마킹. 시각화

In [ ]:
def outlier_comp(data):
#     c_data= concat(data)
    ifor=model(data)
    scores_f= -ifor.decision_function(data)
    d_scatter(scores_f)
    return scores_f

In [ ]:
def min_score(score): #outlier score 최소인 20개 값 추출
    hi=score.tolist()
    hi.sort()
    a=hi[:20]
    return a

In [ ]:
def top_review(score,value,review): #Outlit score 최소인 20개 리뷰 확인
    index = []
    top20_review=[]
    score=score.tolist()
    for i in range(len(value)):
        index.append(score.index(value[i]))
    
    for ind in index:
        top20_review.append(review[ind])
        
    return top20_review

In [ ]:
#이상치 점수#
score_s=outlier_comp(r_embedding) 

In [ ]:
def min_score(score): #outlier score 최소인 20개 값 추출
    hi=score.tolist()
    hi.sort()
    a=hi[:20]
    return a

In [ ]:
def top_review(score,value,review): #Outlit score 최소인 20개 리뷰 확인
    index = []
    top20_review=[]
    score=score.tolist()
    for i in range(len(value)):
        index.append(score.index(value[i]))
    
    for ind in index:
        top20_review.append(review[ind])
        
    return top20_review

In [ ]:
def index(score,value): #Outlit score 최소인 20개 리뷰 확인
    
    index = []
    top20_review=[]
    score=score.tolist()
    for i in range(len(value)):
        index.append(score.index(value[i]))
           
    return index

In [ ]:
min_value=min_score(score_s)
# top=top_review(min_score,)

In [ ]:
# min_value=min_score(score_s)
ind = index(score_s,min_value)

In [ ]:
ind

In [ ]:
def top20_review_check(df): #대중적인 리뷰 20개 확인
    re=df['리뷰'].tolist()
    top_review(score_s,min_value,re)

In [ ]:
final_result

In [ ]:
for x in ind:
    final_result['clustering'][x]=1

In [ ]:
final_result

In [ ]:
def df_to_list(df):
    review=final_result['리뷰'].tolist()
    accuracy=final_result['정확도'].tolist()
    result=final_result['결과'].tolist()
    # score=final_result['평점'].tolist()
    reviewer=final_result['작성자'].tolist()
    date=final_result['날짜'].tolist()
    movie_id=final_result['movie_id'].tolist()
    clustering=final_result['clustering'].tolist()
    return review, accuracy, result, reviewer, date, movie_id, clustering

In [ ]:
def clustering_review(df):
    review=df['리뷰'].tolist()
    df['clustering']=0
    sentence_embedding = sister.MeanEmbedding(lang="file",fasttextfile="cc.ko.100.bin")
    r_embedding=review_embedding(review)
    score_s=outlier_comp(r_embedding) 
    min_value=min_score(score_s)
    ind = index(score_s,min_value,review)
    
    #클러스터링 컬럼 생성
    for x in ind:
        final_result['clustering'][x]=1
    clustering=final_result['clustering'].tolist()
    return clustering

In [ ]:
def db_connect(final_result):
    review, accuracy, result, reviewer, date, movie_id, clustering = df_to_list(final_result)

    conn = pymysql.connect( #DB연결
    host='localhost',
    user='root',
    password='1234',
    db='movie_review',
    charset='utf8' 
    )
    total = len(review) #리뷰 개수
    curs = conn.cursor()
    sql = "insert into reviews(review_id, review_movie_id, review_contents, review_accuracy, review_result, review_reviewer, review_date, review_clustering) values(%s,%s,%s,%s,%s,%s,%s,%s)"
    for i in range(total):
        curs.execute(sql, (i+1 ,movie_id[i], review[i], accuracy[i], result[i], reviewer[i], date[i], clustering[i]))
        print('ok')    
        conn.commit()

In [ ]:
db_connect(final_result)